In [5]:
import cv2
import numpy as np
import os
from PIL import ImageGrab, Image
import os.path as osp
from ultralytics import YOLO

# load model
model = YOLO("runs/detect/train18/weights/best.pt")

# 화면 크기 설정
screen_size = (1280, 720)
# 중앙 300x300 영역 설정
center_size=300
#저장 경로 디폴트셋팅
train_path = "images/"
label_path = "labels/"

# 이미지 저장할 폴더 생성
if not osp.isdir(train_path):
    os.makedirs(train_path)

# 텍스트 파일 저장할 폴더 생성
if not osp.isdir(label_path):
    os.makedirs(label_path)

녹화

In [3]:
# 녹화할 파일명과 코덱 설정
output_file = 'center_screen_recording.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

center_x = (screen_size[0] - center_size) // 2
center_y = (screen_size[1] - center_size) // 2
bbox = (center_x, center_y, center_x + center_size, center_y + center_size)
# 비디오 녹화 객체 생성
out = cv2.VideoWriter(output_file, fourcc, 20.0, (center_size, center_size))
while True:
    # 현재 화면 캡쳐
    img = ImageGrab.grab(bbox=bbox)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # 화면에 표시 및 파일에 저장
    img=np.array(img)
    cv2.imshow('Center Screen Recording', img)
    out.write(img)
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break
# 사용한 자원 해제
out.release()
cv2.destroyAllWindows()

In [ ]:
heroes_num = {'mauga': '1' , 'ana' : '2', 'ashe' : '3', 'baptiste' : '4', 'bastion' : '5', 'brigitte' : '6', 'cassidy' : '7',
                  'd.va' : '8', 'doomfist' : '9', 'echo' : '10', 'genji' : '11', 'hanzo' : '12', 'illari' : '13', 'junkerqeen' : '14',
                  'junkrat' : '15', 'kiriko' : '16', 'lifeweaver' : '17', 'lucio' : '18', 'mei' : '19', 'mercy' : '20', 'moira' : '21',
                  'orisa' : '22', 'pharah' : '23', 'ramattra' : '24', 'reaper' : '25', 'reinhardt' : '26', 'roadhog' : '27', 'sigma' : '28',
                  'sojourn' : '29', 'soldier:76' : '30', 'sombra' : '31', 'symmetra' : '32', 'torbjorn' : '33', 'tracer' : '34',
                  'widowmaker' : '35', 'winston' : '36', 'wreckingball' : '37', 'zarya' : '38', 'zenyatta' : '39'} 

# 영웅 이름 리턴하는 함수
def get_hero_num(key):
    global heroes_num
    if key in heroes_num:
        return heroes_num[key]
    else:
        return "해당하는 영웅을 찾을 수 없습니다."

In [4]:
# 박스의 w/2,h/2 는 size의 크기와 같도록 함
size=5

# 폴더 내 파일 개수 확인
def file_count(path):
    global count 
    folder_path = path
    file_list = os.listdir(folder_path)
    count = len(file_list)
    return count
count = file_count(train_path)

# 클릭 좌표 관련 변수
point=[]
click_coordinate = []
pointed=False
while_rec=False

# 캐릭터별 번호
cls={"봇": 0,"트레": 1}

# 마우스 이벤트
def mouse_callback(event, x, y, flags, param):
    global point, img, count, pointed, while_rec
    global size

    if event == cv2.EVENT_LBUTTONDOWN:
        point = [x,y]
        tempimg=img.copy()

        if while_rec==True:
            pointed=True
            cv2.putText(tempimg, f"stop count:{count}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
            cv2.rectangle(tempimg, (point[0]-size, point[1]-size), (point[0]+size, point[1]+size), (0, 0, 255), 1)
            cv2.imshow("win",tempimg)
    
    # 마우스 휠에 따라 사각형 크기 조절
    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            size += 1
        elif size > 1:
            size -= 1

# 마우스 클릭 좌표 기준 사각형 그리기
def make_rectangle():
    global img, pointed
    global point, while_rec, count
    while_rec=True

    key=cv2.waitKey(0)

    # 스페이스 입력 시 영상 정지 및 마우스 클릭 좌표 정보를 텍스트 파일에 저장
    if key == ord(" "):
        if pointed==True:
            count = file_count()
            
            # 텍스트 파일 열기
            text = open_file(label_path + str(count) + ".txt", "w")

            print(f"{point[0]},{point[1]}")

            # 텍스트 파일에 좌표값 저장
            text.write(f"{cls['봇']} {float(point[0])/center_size} {float(point[1])/center_size} {float(size)*2/center_size} {float(size)*2/center_size}")

            # faces/count.jpg 로 저장
            cv2.imwrite(train_path + str(count) + '.jpg', img)
            
            # 텍스트 파일 닫기
            text.close()
            pointed=False
    while_rec=False

# 파일 열기 함수
def open_file(filename, mode):
    try:
        return open(filename, mode)
    except FileNotFoundError:
        return open(filename, "w")

D: 60프레임 이후로 스킵
A: 60프레임 이전으로 스킵
Space: 화면 정지
좌클릭: 중앙점 생성, 정사각형 생성
횔업&다운: 정사각형 크기 늘리고 줄이고
E&R: 사각형 폭 늘리고 줄이기

In [3]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)
ret,img=readvideo.read()

while ret:
    key = cv2.waitKey(33)

    # 프레임 조정
    if key == ord("d"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) + 60)
    elif key == ord("a"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) - 60)

    # 사각형 생성
    elif key == ord(" "):
        make_rectangle()

    # 영상 종료
    elif key == ord("q"):
        break
    
    ret,img=readvideo.read()

    cv2.imshow("win", img)

readvideo.release()
cv2.destroyAllWindows()

212,133
56,182
129,71


In [14]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
ret,img=readvideo.read()

while ret:
    key = cv2.waitKey(1)

    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
            result = r.boxes.xywh[0].cpu().numpy()
            result = result.tolist()
            result = [round(element) for element in result]
            print(result)
    
            x, y, w, h = result
            cv2.rectangle(img, (x - w, y - h), (x + w, y + h), (0, 0, 255), 2)        
     

    cv2.imshow("win", img)

    ret,img=readvideo.read()

    # 영상 종료
    if key == ord("q"):
        break
readvideo.release()
cv2.destroyAllWindows()


0: 320x320 1 bot, 10.8ms
Speed: 1.0ms preprocess, 10.8ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 9.1ms
Speed: 3.6ms preprocess, 9.1ms inference, 4.1ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]



0: 320x320 1 bot, 13.3ms
Speed: 4.4ms preprocess, 13.3ms inference, 7.5ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 7.6ms
Speed: 1.1ms preprocess, 7.6ms inference, 4.7ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 7.1ms
Speed: 1.3ms preprocess, 7.1ms inference, 5.9ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 8.8ms
Speed: 1.0ms preprocess, 8.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 5.5ms
Speed: 3.3ms preprocess, 5.5ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 6.3ms
Speed: 1.3ms preprocess, 6.3ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 8.9ms
Speed: 1.1ms preprocess, 8.9ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 10.5ms


In [3]:
results=model("./images/248.JPG")

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    bb=boxes.xywh[0].cpu().numpy()
    bb=bb.tolist()
    print(bb)
    masks = result.masks  # Masks object for segmentation masks outputs
    print(masks)
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


image 1/1 c:\Users\Administator\Desktop\PJT_MCL\images\248.JPG: 320x320 1 bot, 5.6ms
Speed: 2.0ms preprocess, 5.6ms inference, 6.6ms postprocess per image at shape (1, 3, 320, 320)
[126.93994140625, 74.19407653808594, 34.51824188232422, 33.23259735107422]
None


In [ ]:
# load libraries
from ultralytics import YOLO
from PIL import Image

# load model
model = YOLO("./runs/detect/train18/weights/best.pt")

# inference
image_path = "./images/248.JPG"

results=model(image_path)

# list 형태에서 tensor -> numpy 변환
for r in results:
    result = r.boxes.xywh
    result = result.numpy()

arr_int = result.astype(int)

# 얼굴 영역 사각형 출력
import cv2

src = cv2.imread(image_path)

for x, y, w, h in arr_int:
    cv2.rectangle(src, (x - w, y - h), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow("test", src)
cv2.waitKey()
cv2.destroyAllWindows()